# Setup

In [1]:
%reload_ext autoreload

In [2]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
np.set_printoptions(precision=3)
import scipy.stats
import einops


from functools import partial
from collections import namedtuple
import itertools
from itertools import repeat
from time import time

import chex
import jax
import jax.random as jr
import jax.numpy as jnp
from jax import vmap, grad, jit, lax
from jax import numpy as jnp
import jax.scipy as jsp

from flax.core import freeze, unfreeze
from flax import linen as nn
import flax

import jaxopt
import optax


from PIL import Image


#jax.config.update("jax_enable_x64", False)



In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds


import torch
from torch.utils.data import TensorDataset
import torchvision.transforms as T

In [3]:
import os 
cpu_count = os.cpu_count()
print(cpu_count)

# Run jax on multiple CPU cores
# https://github.com/google/jax/issues/5506
# https://stackoverflow.com/questions/72328521/jax-pmap-with-multi-core-cpu
import os 
#os.environ["XLA_FLAGS"] = '--xla_force_host_platform_device_count=90'

import jax
print(jax.devices())

96
[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0), TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1), TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0), TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1), TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0), TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1), TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0), TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]


# Import library code

In [4]:
# https://github.com/probml/probml-utils
import probml_utils
from probml_utils.mlp_flax import MLPNetwork, NeuralNetClassifier
print(MLPNetwork)

2022-11-11 19:09:50.715904: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2022-11-11 19:09:50.750871: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-11 19:09:51.498447: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2022-11-11 19:09:51.498617: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib


<class 'probml_utils.mlp_flax.MLPNetwork'>


In [5]:

import tta
from tta.utils import *
print(Dataset)

from tta.datasets import *
print(MultipleDomainDataset)

from tta.datasets.chexpert import *
print(MultipleDomainCheXpert)


<class 'tta.utils.Dataset'>
<class 'tta.datasets.MultipleDomainDataset'>
<class 'tta.datasets.chexpert.MultipleDomainCheXpert'>


# Load pre-computed data matrix

In [6]:

from pathlib import Path
root = '/home/kpmurphy/data/CheXpert'
root = Path(root)

data = np.load(root / 'data_matrix.npz', allow_pickle=True)
print(data.files)
print(data['features'].shape, data['attributes'].shape)

['features', 'attributes', 'columns', 'uniques']
(190499, 1376) (190499, 19)


In [16]:

disease_ndx = np.where(data['columns'] == 'EFFUSION')[0][0]
Yall = data['attributes'][:, disease_ndx]

gender_ndx = np.where(data['columns'] == 'GENDER')[0][0]
Zall = data['attributes'][:, gender_ndx]

# diease labels
# 0 = no mention  
# 1 = positive   
# 2 = uncertain  
# 3 = negative  
# gender: 0=female, 1= male, 2=unknown
print('number of unknown gender = ', np.sum(Zall==2))
print('number of unknown disease = ', np.sum( (Yall==0) | (Yall==2) ) )
remove = (Yall==0) | (Yall==2) | (Zall==2)
ndx = np.where(remove==True)[0]
nremove = len(ndx)
print('removing ', nremove, ' rows')
keep = ~remove

X = data['features'][keep]

Y = Yall[keep]
pos = (Y==1); neg = (Y==3)
Y[pos]=1; Y[neg]=0

Z  = Zall[keep]

print(X.shape, Y.shape, Z.shape)


number of unknown gender =  1
number of unknown disease =  34898
removing  34898  rows
(155601, 1376) (155601,) (155601,)


# Fit logistic regression with sklearn

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import sklearn
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline

In [28]:


from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


N_train  = X_train.shape[0]
N_test  = X_test.shape[0]
print([N_train, N_test])

classifier = Pipeline([
        ('standardscaler', StandardScaler()),
        #('poly', PolynomialFeatures(degree=2)), 
        ('logreg', LogisticRegression(random_state=0, max_iter=500, C=10, solver='sag', multi_class='multinomial'))
])

classifier = LogisticRegression(random_state=0, max_iter=500, C=10, solver='sag', multi_class='multinomial')


[111925, 27982]


In [29]:


#N = 100
N  = N_train
XX = X_train[:N]
YY = Y_train[:N]




In [30]:
%%time
classifier.fit(XX, YY)

In [27]:

probs = classifier.predict_proba(X_test)

y_pred = jnp.argmax(probs, axis=1)
y_pred2 = classifier.predict(X_test)
assert np.allclose(y_pred, y_pred2)

y_true = Y_test
acc = sklearn.metrics.accuracy_score(y_true, y_pred)
print(acc)



0.6214352083482239


# Fit logistic regression with flax

In [17]:


from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

N_train  = X_train.shape[0]
N_test  = X_test.shape[0]
print([N_train, N_test])





[124480, 31121]


In [22]:
#N = 1000 # use subset of data
N  = N_train # use all data

XX = X_train[:N]
nclasses = 2
YY = Y_train[:N]


In [29]:
nhidden = () + (nclasses,) # set nhidden() to get logistic regression
network = MLPNetwork(nhidden)
key = jr.PRNGKey(0)
opt = optax.adamw(1e-3) 
#opt = 'adam+warmup'
mlp = NeuralNetClassifier(network, key, nclasses, l2reg=1e-3, standardize=False,
        batch_size=512, num_epochs=30, print_every=5, optimizer=opt)  


In [30]:
%%time

mlp.fit(XX, YY)

epoch 0, train loss 0.493, train accuracy 0.762
epoch 5, train loss 0.460, train accuracy 0.785
epoch 10, train loss 0.456, train accuracy 0.786
epoch 15, train loss 0.456, train accuracy 0.787
epoch 20, train loss 0.455, train accuracy 0.787
epoch 25, train loss 0.455, train accuracy 0.788
CPU times: user 5min 28s, sys: 1min, total: 6min 28s
Wall time: 4min 5s


In [31]:

probs = mlp.predict(X_train)
y_pred = jnp.argmax(probs, axis=1)
acc = jnp.mean(Y_train == y_pred)
print('train accuracy', acc)

probs = mlp.predict(X_test)
y_pred = jnp.argmax(probs, axis=1)
acc = jnp.mean(Y_test == y_pred)
print('test accuracy', acc)

train accuracy 0.7917176
test accuracy 0.7879246


# Make shifted datasets for each domain (WIP)


In [ ]:
root = Path("/home/kpmurphy/data/CheXpert")
dataset_y_column = "EFFUSION"
dataset_z_column = "GENDER"
dataset_use_embedding = True
train_domains_set = [9]
target_domain_count = 512

import random
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
key = jax.random.PRNGKey(seed)
generator = torch.Generator().manual_seed(seed)


dataset = MultipleDomainCheXpert(root, generator, dataset_y_column, dataset_z_column, dataset_use_embedding, train_domains_set, target_domain_count)